# Part I. ETL Pipeline for Pre-Processing the Files


## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES


#### Import Python packages

In [62]:
import csv
import glob
import pandas as pd
import cassandra
import os

#### Creating list of filepaths to process original event csv data files

In [63]:
print(os.getcwd())

/Users/shaikirfankareemali/PycharmProjects/Data-Modelling-with-Cassandra


In [64]:
file_path = os.getcwd() + '/event_data'
print(file_path)

/Users/shaikirfankareemali/PycharmProjects/Data-Modelling-with-Cassandra/event_data


In [65]:
for root, dirs, files in os.walk(file_path):
    file_path_list = glob.glob(os.path.join(root, '*'))

# print(len(file_path_list))

#### Processing the files to create the data file csv that will be used for Apache Cassandra tables.

In [66]:
combined_csv = pd.concat([pd.read_csv(file) for file in file_path_list])
combined_csv = combined_csv[combined_csv.artist.notnull()]
combined_csv = combined_csv[['artist','firstName','gender','itemInSession','lastName','length','level','location','sessionId','song','userId']]
combined_csv.to_csv("event_datafile_new.csv", index=False, encoding="utf-8")

# Part II. Complete the Apache Cassandra coding portion of your project.

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns:
- artist
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [67]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

In [68]:
session.execute("""
                CREATE KEYSPACE IF NOT EXISTS sparkify
                WITH REPLICATION = {'class' : 'SimpleStrategy',
                    'replication_factor' : 1 }
                """)

In [69]:
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [70]:
# Query-1
# Give the artist, song title and length in the music app history that was heard during sessionId = 338 and itemInSession = 4

session.execute("""
                CREATE TABLE IF NOT EXISTS song_sessions(
                    session_id int,
                    item_in_session int,
                    artist text,
                    song_title text,
                    song_length float,
                    PRIMARY KEY(session_id, item_in_session)
                )
                """)

In [72]:
event_data_path = 'event_datafile_new.csv'

with open(event_data_path, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO song_sessions(session_id, item_in_session, artist, song_title, song_length)"
        query += " VALUES (%s, %s, %s, %s, %s)"
        artist, first_name, gender, itemInSession, last_name, length, level, location, sessionId, song, userId = line
        session.execute(query,(int(sessionId), int(itemInSession), artist, song, float(length)))
print("INSERTED SUCCESSFULLY!")

INSERTED SUCCESSFULLY!


In [73]:
result = session.execute("SELECT artist, song_title, song_length FROM song_sessions WHERE session_id = 338 AND item_in_session = 4")

print("artist, song_title, song_length")

for row in result:
    print(row.artist,", ", row.song_title,", ", row.song_length)

artist, song_title, song_length
Faithless ,  Music Matters (Mark Knight Dub) ,  495.30731201171875


In [74]:
# Query-2
# Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

session.execute("""
                CREATE TABLE IF NOT EXISTS songs_users_sessions(
                    user_id int,
                    session_id int,
                    artist_name text,
                    song_title text,
                    item_in_session int,
                    first_name text,
                    last_name text,
                    PRIMARY KEY((user_id, session_id), item_in_session)
                )
                """)

In [75]:
event_data_path = 'event_datafile_new.csv'

with open(event_data_path, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query2 = "INSERT INTO songs_users_sessions(user_id, session_id, artist_name, song_title, item_in_session, first_name, last_name) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        artist, first_name, gender, itemInSession, last_name, length, level, location, sessionId, song, userId = line
        session.execute(query2,(int(float(userId)), int(sessionId), artist, song, int(itemInSession), first_name, last_name))
print("INSERTED SUCCESSFULLY!")

INSERTED SUCCESSFULLY!


In [76]:
result2 = session.execute("SELECT artist_name, song_title, first_name, last_name "
                          "FROM songs_users_sessions "
                          "WHERE user_id = 10 AND session_id = 182"
                          "ORDER BY item_in_session")

print("Artist Name, Song Title, First Name, Last Name")
print()
for row in result2:
    print(row.artist_name, ", ", row.song_title, ", ", row.first_name, ", ", row.last_name)

Artist Name, Song Title, First Name, Last Name

Down To The Bone ,  Keep On Keepin' On ,  Sylvie ,  Cruz
Three Drives ,  Greece 2000 ,  Sylvie ,  Cruz
Sebastien Tellier ,  Kilometer ,  Sylvie ,  Cruz
Lonnie Gordon ,  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) ,  Sylvie ,  Cruz


In [77]:
# Query-3
# Give me every username (first and last) in my music app history who listened to the song 'All Hands Against His Own'

session.execute("""
                CREATE TABLE IF NOT EXISTS songs_users(
                    song_title text,
                    user_id int,
                    first_name text,
                    last_name text,
                    PRIMARY KEY(song_title, user_id)
                )
                """)

In [78]:
with open(event_data_path, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)

    for line in csvreader:
        query3 = "INSERT INTO songs_users(song_title, user_id, first_name, last_name) VALUES (%s, %s, %s, %s)"
        artist, first_name, gender, itemInSession, last_name, length, level, location, sessionId, song, userId = line
        session.execute(query3,(song, int(float(userId)), first_name, last_name))
print("INSERTED SUCCESSFULLY!")

INSERTED SUCCESSFULLY!


In [79]:
result3 = session.execute("SELECT first_name, last_name FROM songs_users WHERE song_title = 'All Hands Against His Own'")

for row in result3:
    print(row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [80]:
session.execute("DROP TABLE IF EXISTS song_sessions")
session.execute("DROP TABLE IF EXISTS songs_users_sessions")
session.execute("DROP TABLE IF EXISTS songs_users2")

### Close the session and cluster connection

In [81]:
session.shutdown()
cluster.shutdown()